Este es un ejemplo extraído del tutorial "Convolutional Neural 
Networks in Python with Keras" de la página "Datacamp" por Aditya Sharma. 

https://www.datacamp.com/community/tutorials/convolutional-neural-networks-python

<hr>  


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

<h1>REDES NEURONALES CONVOLUCIONALES CON KERAS</h1>
El Dataset Fashion-MNIST se encuentra disponible en la librería 
datasets de Keras y contiene imágenes de los artículos de Zalando 
(tienda de moda en línea alemana). Las imágenes son 
escala de grises de 28x28 y corresponden a 70.000 productos 
de moda de 10 categorías diferentes (7.000 imágenes por categoría). 
El conjunto de entrenamiento tiene 60.000 imágenes y el conjunto 
de prueba tiene 10.000 imágenes.

In [ ]:
from tensorflow.keras.datasets import fashion_mnist

Las imágenes de entrenamiento y prueba, junto con sus etiquetas 
correspondientes, se almacenan en las variables <i>train_images, 
train_labels, test_images</i> y <i>test_labels</i> respectivamente.

In [ ]:
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [ ]:
print(f"Training set shape: ({train_images.shape}, {train_labels.shape})")

print(f"Test set shape: ({test_images.shape},{test_labels.shape})")

In [ ]:
classes = np.unique(train_labels)
nClasses = len(classes)
print('Total number of outputs : ', nClasses)
print('Output classes : ', classes)

In [ ]:
label_dict = {
 0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot',
}

In [ ]:
plt.figure(figsize=[5,5])

plt.subplot(121)
curr_img = train_images[0]
curr_lbl = train_labels[0]
plt.imshow(curr_img, cmap='gray')
plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")

plt.subplot(122)
curr_img = test_images[0]
curr_lbl = test_labels[0]
plt.imshow(curr_img, cmap='gray')
plt.title("(Label: " + str(label_dict[curr_lbl]) + ")");

Es necesario realizar un preprocesamiento de los datos para poder 
utilizarlos en el modelo:
<ol>
    <li>Convertir cada imagen de 28x28 en una matriz de 28x28x1.</li>
</ol>

In [ ]:
train_images = train_images.reshape(-1, 28,28, 1)
test_images = test_images.reshape(-1, 28,28, 1)
train_images.shape, test_images.shape

<ol start="2">
    <li>Convertir los datos de tipo int8 a float32.</li>
</ol>

In [ ]:
train_images = train_images.astype('float32')
test_images = test_images.astype('float32')

<ol start="3">
    <li>Re-escalar los valores de los pixeles del rango 0-255 a 0-1.</li>
</ol>

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

<ol start="4">
    <li>Convertir las etiquetas de clase en un vector de codificación 
        <i>one-hot</i>: Esto es necesario porque 
        los algoritmos de aprendizaje automático no pueden 
        trabajar directamente con datos categóricos. En la codificación 
        one-hot, se convierten los datos categóricos en un vector 
        de números. Se genera una columna
        para cada categoría o clase. Por lo que, cada vector de 
        codificación consiste de todos ceros excepto por un 1 que 
        indica a la clase que corresponde.</li>
</ol>

In [ ]:
from tensorflow.keras.utils import to_categorical

train_labels_oh = to_categorical(train_labels)
test_labels_oh = to_categorical(test_labels)

print('Original label:', train_labels[0])
print('After conversion to one-hot:', train_labels_oh[0])

<ol start="5">
    <li>Dividir el set de datos de entrenamiento en datos de entrenamiento 
    y validación.</li>
</ol>

In [ ]:
from sklearn.model_selection import train_test_split
train_X, valid_X, train_label,valid_label = train_test_split(train_images, train_labels_oh, test_size = 0.2, random_state = 13)

La red neuronal a aplicar consiste de:
<ul>
    <li>Una primera capa con 32 filtros 3x3.</li>
    <li>Una segunda capa con 64 filtros 3x3.</li>
    <li>Una tercera capa con 128 filtros 3x3.</li>
    <li>3 capas de tipo max-pooling de tamaño 2x2 luego 
        de cada capa convolucional.</li>
    <li>Una capa densa de 128 neuronas.</li>
    <li>Una capa de salida con 10 neuronas.</li>
</ul>

In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

<b>Batch size</b> es el número de ejemplos de entrenamiento que son 
utilizados en cada iteración.

<b>Epoch</b> es un término para indicar el número de veces que todo
el set de datos de entrenamiento atraviesa el algoritmo. Mientras 
que <b>iteración</b> hace referencia a cuando un batch de datos atraviesa el 
algoritmo. En el caso de que el tamaño de batch sea igual al tamaño del 
set de datos de entrenamiento, el número de epochs y de iteraciones 
sería el mismo.


In [ ]:
batch_size = 64
epochs = 20
num_classes = 10

In [ ]:
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1),padding='same'))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))                 
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='relu'))                
fashion_model.add(Dense(num_classes, activation='softmax'))

In [ ]:
fashion_model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])

In [ ]:
fashion_model.summary()

In [ ]:
fashion_train = fashion_model.fit(train_X, train_label, batch_size = batch_size, epochs = epochs, verbose = 1,validation_data = (valid_X, valid_label))

In [ ]:
test_eval = fashion_model.evaluate(test_images, test_labels_oh, verbose=0)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
accuracy = fashion_train.history['accuracy']
val_accuracy = fashion_train.history['val_accuracy']
loss = fashion_train.history['loss']
val_loss = fashion_train.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

<h2>APLICANDO DROPOUT</h2>

In [ ]:
batch_size = 64
epochs = 20
num_classes = 10

In [ ]:
fashion_model = Sequential()
fashion_model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding='same',input_shape=(28,28,1)))
fashion_model.add(MaxPooling2D((2, 2),padding='same'))
fashion_model.add(Dropout(0.25))
fashion_model.add(Conv2D(64, (3, 3), activation='relu',padding='same'))
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.25))
fashion_model.add(Conv2D(128, (3, 3), activation='relu',padding='same'))                 
fashion_model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
fashion_model.add(Dropout(0.4))
fashion_model.add(Flatten())
fashion_model.add(Dense(128, activation='relu'))        
fashion_model.add(Dropout(0.3))
fashion_model.add(Dense(num_classes, activation='softmax'))

In [ ]:
fashion_model.summary()

In [ ]:
fashion_model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=['accuracy'])

In [ ]:
fashion_train_dropout = fashion_model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

In [ ]:
test_eval = fashion_model.evaluate(test_images, test_labels_oh, verbose=0)

In [ ]:
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])

In [ ]:
accuracy = fashion_train_dropout.history['accuracy']
val_accuracy = fashion_train_dropout.history['val_accuracy']
loss = fashion_train_dropout.history['loss']
val_loss = fashion_train_dropout.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
predicted_classes = fashion_model.predict(test_images)


In [ ]:
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)

In [ ]:
predicted_classes.shape, test_labels.shape

In [ ]:
correct = np.where(predicted_classes==test_labels)[0]
print (f"Found {len(correct)} correct labels") 
for i, correct in enumerate(correct[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_images[correct].reshape(28,28), cmap='gray')
    plt.title(f"Predicted {predicted_classes[correct]}, Class {test_labels[correct]}")
    
plt.tight_layout();

In [ ]:
incorrect = np.where(predicted_classes != test_labels)[0]
print (f"Found {len(incorrect)} incorrect labels")
for i, incorrect in enumerate(incorrect[:9]):
    plt.subplot(3,3,i+1)
    plt.imshow(test_images[incorrect].reshape(28,28), cmap='gray')
    plt.title(f"Predicted {predicted_classes[incorrect]}, Class {test_labels[incorrect]}")
    
plt.tight_layout();

In [ ]:
from sklearn.metrics import classification_report
target_names = [f"Class {i}" for i in range(num_classes)]
print(classification_report(test_labels, predicted_classes, target_names=target_names))